In [13]:
import numpy as np
import optuna
import pandas as pd
import xgboost as xgb
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

from lifelines import KaplanMeierFitter

from sklearn.metrics import f1_score


In [14]:
def transform_survival_probability(df, time_col='efs_time', event_col='efs'):
    kmf = KaplanMeierFitter()
    df_ = df.loc[(df[time_col] < 24) | (df[event_col] == 0)]
    kmf.fit(df_[time_col], df_[event_col])
    y = kmf.survival_function_at_times(df[time_col]).values
    plt.hist(df.efs_time[df[event_col] == 0], bins=150, alpha=0.5, label="Event")
    plt.show()
    #y = y / (y.max() - y.min())
    # y = np.log(1 + np.log(y / (1 - y)))
    return y


# not used so far
def preprocess_data(df):
    df[CATEGORICAL_VARIABLES] = df[CATEGORICAL_VARIABLES].fillna("Unknown")
    df[OTHER_NUMERICAL_VARIABLES] = df[OTHER_NUMERICAL_VARIABLES].fillna(df[OTHER_NUMERICAL_VARIABLES].median())

    return df

# not used so far
def features_engineering(df):
    # Change year_hct to relative year from 2000
    df['year_hct'] = df['year_hct'] - 2000
    
    return df


In [15]:
def run(hparams=None):
    FOLDS, oof_xgb, pred_xgb, train = make_predictions(hparams)

    # COMPUTE AVERAGE TEST PREDS
    pred_xgb /= FOLDS

    y_true = train[["ID", "efs", "efs_time", "race_group"]].copy()
    y_pred = train[["ID"]].copy()
    y_pred["prediction"] = oof_xgb
    m = score_f(train.copy(), y_pred.copy(), "ID")
    plt.hist(oof_xgb[train.efs==1], bins=np.linspace(-3, 3, 200), alpha=0.5, label="Event")
    plt.hist(oof_xgb[train.efs==0], bins=np.linspace(-3, 3, 200), alpha=0.5, label="No event")
    plt.legend()
    plt.show()
    print(f"\nOverall CV for XGBoost KaplanMeier =", m)
    return pred_xgb


def make_predictions(hparams):
    if hparams is None:
        hparams = dict(
            max_depth=6,
            colsample_bytree=0.5,
            subsample=0.8,
            n_estimators=3000,
            learning_rate=0.01,
            min_child_weight=40,
            gamma=1,
            eta=0.0,
            reg_lambda=0.1,
            reg_alpha=0.1,
            eps=2e-2,
            eps_mul=1.01,
            pos_shift=0.2
        )
    FEATURES, test, train, y = prepare_data(hparams.pop('eps'), hparams.pop('eps_mul'))
    FOLDS = 5
    kf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
    oof_xgb = np.zeros(len(train))
    pred_xgb = np.zeros(len(test))
    pos_shift = hparams.pop('pos_shift')
    for i, (train_index, test_index) in enumerate(kf.split(train, train.race_group)):
        print("#" * 25)
        print(f"### Fold {i + 1}")
        print("#" * 25)
        x_test, x_train, x_valid, y_train, y_valid = _prepare_training_data(
            FEATURES, test.copy(), test_index, train.copy(), train_index, y.copy(), pos_shift=pos_shift
        )
        hparams.update(
            dict(
                objective='reg:pseudohubererror',
                max_cat_to_onehot=10,
                device="cuda",
                enable_categorical=True,
                random_state=42,
                monotone_constraints={
                    # 'comorbidity_score': 1,
                    # 'hla_match_c_high': -1,
                    # 'hla_high_res_10': -1,
                    'hla_high_res_6': -1,
                    'hla_high_res_8': -1,
                    # 'hla_low_res_10': -1,
                    'hla_low_res_6': -1,
                    # 'hla_low_res_8': -1,
                    'hla_match_a_high': -1,
                    # 'hla_match_a_low': -1,
                    # 'hla_match_b_high': -1,
                    'hla_match_drb1_low': -1,
                    'hla_match_c_low': -1,
                    # 'hla_match_c_high': -1,
                    # 'donor_age': -1,
                    # 'hla_match_drb1_high': -1,
                    'hla_match_dqb1_low': -1,
                    'hla_nmdp_6': -1,
                    # 'karnofsky_score': -1,

                }
            )
        )
        print(hparams['n_estimators'])
        model_xgb = XGBRegressor(
            **hparams
        )
        tt = train.loc[train_index]

        array = np.array([.95 if x else 1 for x in ((tt.efs_time < 24) & (tt.efs == 0)).values])
        array /= np.array([1.3 if x else 1 for x in tt.efs_time > 36])
        model_xgb.fit(
            x_train, y_train,
            eval_set=[(x_train, y_train), (x_valid, y_valid)],
            verbose=100,
            sample_weight=array
        )

        # INFER OOF
        oof_xgb[test_index] = model_xgb.predict(x_valid)
        # INFER TEST
        pred_xgb += model_xgb.predict(x_test)
    return FOLDS, oof_xgb, pred_xgb, train


def _prepare_training_data(FEATURES, test, test_index, train, train_index, y, pos_shift=0.1):
    y[train.efs == 0] = y[train.efs == 1].min() - pos_shift
    std = np.std(y[train_index])
    x_train = train.loc[train_index, FEATURES].copy()
    y_train = y[train_index] / std
    x_valid = train.loc[test_index, FEATURES].copy()
    y_valid = y[test_index] / std
    x_test = test[FEATURES].copy()
    # ind = (train.loc[train_index].efs_time < 24) | (train.loc[train_index].efs == 0)
    # x_train = x_train[ind]
    # y_train = y_train[ind]

    le = LabelEncoder()
    val = train.loc[test_index]
    return x_test, x_train, x_valid, y_train, y_valid



def prepare_data(eps=2e-2, eps_mul=1.1):
    test = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")
    test = add_features(test)
    train = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/train.csv")
    train = add_features(train)
    train["y"] = transform_survival_probability(train, time_col='efs_time', event_col='efs')
    RMV = ["ID", "efs", "efs_time", "y"]
    FEATURES = [c for c in train.columns if not c in RMV]

    CATS = []
    for c in FEATURES:
        if train[c].dtype == "object":
            CATS.append(c)
            train[c] = train[c].fillna("NAN")
            test[c] = test[c].fillna("NAN")
    print(f"In these features, there are {len(CATS)} CATEGORICAL FEATURES: {CATS}")
    print(f"In these features, there are {len(CATS)} NUMERICAL FEATURES: {sorted([f for f in FEATURES if f not in CATS])}")
    combined = pd.concat([train, test], axis=0, ignore_index=True)
    # print("Combined data shape:", combined.shape )
    # LABEL ENCODE CATEGORICAL FEATURES
    for c in FEATURES:

        # LABEL ENCODE CATEGORICAL AND CONVERT TO INT32 CATEGORY
        if c in CATS:
            combined[c], _ = combined[c].factorize()
            combined[c] -= combined[c].min()
            combined[c] = combined[c].astype("int32")
            combined[c] = combined[c].astype("category")

        # REDUCE PRECISION OF NUMERICAL TO 32BIT TO SAVE MEMORY
        else:
            if combined[c].dtype == "float64":
                combined[c] = combined[c].astype("float32")
            if combined[c].dtype == "int64":
                combined[c] = combined[c].astype("int32")
    train = combined.iloc[:len(train)].copy()
    test = combined.iloc[len(train):].reset_index(drop=True).copy()
    y = train.y.copy()
    # y = np.log(y)
    y = (y - y.min() + eps) / (y.max() - y.min() + eps_mul * eps)
    y = np.log(y / (1 - y))
    plt.hist(y[train.efs == 1], bins=np.linspace(-4, 8, 150), label="Event")
    plt.hist(y[train.efs == 0], bins=np.linspace(-4, 8, 150), label="No event")
    plt.legend()
    plt.show()
    return FEATURES, test, train, y

In [ ]:
pred_xgb = run(None)